PromptTemplate vs ChatPromptTemplate

PromptTemplate은 단순한 메시지를 전달할 때 사용

ChatPromptTemplate은 system, ai, human으로 구분하여 메시지를 전달할 때 사용. 

# PromptTemplate vs ChatPromptTemplate

## PromptTemplate
- **사용 용도**: 단순한 메시지나 질문을 전달할 떄 사용
- **특징**: 역할 구분 없이 단일 텍스트 형태로 작성된 프롬프트로 사용

## ChatPromptTemplate
- **사용 용도**: 'system', 'ai', 'human'과 같은 역할을 구분하여 여러 메시지를 전달할 때 사용
- **특징**: 대화 흐름이나 맥락 설정이 필요한 복잡한 대화에 적합 

In [3]:
# 가장 기본적인 사용 형태

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

# 얼마나 창의적으로 할 것인지 낮을수록 형식적인 답변

chat = ChatOpenAI(
    temperature=0.1
)
template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)
prompt = template.format(country_a="Mexico", country_b="Thailand")
chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [2]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage
chat = ChatOpenAI(
    temperature=0.1
)

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geograpy expert. And you only reply in {language}."),
    ("ai","Ciao, mi chiamo {name}!"),
    ("human","What is the distance between {country_a} and {country_b}. Also what is your name?")
])
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)
chat.predict_messages(prompt)


AIMessage(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

# Chain 형성

- **chain** : 복잡한 작업을 수행할 때 코드를 효율적으로 구성해주는 장치
- **chain** 순서: template | model | parser

In [ ]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
chat = ChatOpenAI(
    temperature=0.1
)    

template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lower case. Do Not reyply with anything else."),
    ("human","{question}")
])

chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items":5,
    "question":"What are the pokemons?"    
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']